# IMPACT paper experiments
### 1. Init
#### 1.1. Import libraries (necessary)

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from liriscat import utils
utils.set_seed(0)
from liriscat import dataset
from liriscat import selectionStrategy
from liriscat import CDM

import logging
import gc
import json
import torch
import pandas as pd
from importlib import reload
import IMPACT

#### 1.2. Set up the loggers (recommended)

In [2]:
utils.setuplogger(verbose = True, log_name="liriscat")

### 2. CDM prediction
#### 2.1. Training and testing, sequential version

In [3]:
import warnings
import numpy as np

gc.collect()
torch.cuda.empty_cache()

reload(utils)
reload(selectionStrategy)
reload(CDM)
reload(dataset)

<module 'liriscat.dataset' from '/home/arthurb/Programmation/liriscat/liriscat/dataset/__init__.py'>

In [13]:
config = utils.generate_eval_config(esc = 'error', valid_metric= 'mi_acc', pred_metrics = ["mi_acc"], profile_metrics = ['doa'], save_params=False, n_query=6, num_epochs=0, batch_size=512)
utils.set_seed(config["seed"])

config["dataset_name"] = "math2"
logging.info(config["dataset_name"])
config['learning_rate'] = 0.02026
config['lambda'] = 1.2e-5
config['d_in'] = 4
config['num_responses'] = 12
#pred_metrics,df_interp = test(config)

CUDA is available. Using GPU.
[INFO 19:41] math2


In [14]:
logging.info(f'#### {config["dataset_name"]} ####')
logging.info(f'#### config : {config} ####')
config['embs_path']='../embs/'+str(config["dataset_name"])
config['params_path']='../ckpt/'+str(config["dataset_name"])

pred_metrics = {m:[] for m in config['pred_metrics']}
profile_metrics = {m:[] for m in config['profile_metrics']}

gc.collect()
torch.cuda.empty_cache()

# Dataset downloading for doa and rm
warnings.filterwarnings("ignore", message="invalid value encountered in divide")
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Concept map format : {question_id : [category_id1, category_id2, ...]}
concept_map = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_concept_map.json', 'r'))
concept_map = {int(k): [int(x) for x in v] for k, v in concept_map.items()}

## Metadata map format : {"num_user_id": ..., "num_item_id": ..., "num_dimension_id": ...}
metadata = json.load(open(f'../datasets/2-preprocessed_data/{config["dataset_name"]}_metadata.json', 'r'))

[INFO 19:42] #### math2 ####
[INFO 19:42] #### config : {'seed': 0, 'dataset_name': 'math2', 'load_params': False, 'save_params': False, 'embs_path': '../embs/', 'params_path': '../ckpt/', 'early_stopping': True, 'esc': 'error', 'verbose_early_stopping': False, 'disable_tqdm': False, 'valid_metric': 'mi_acc', 'learning_rate': 0.02026, 'batch_size': 512, 'num_epochs': 0, 'eval_freq': 1, 'patience': 30, 'device': device(type='cuda'), 'lambda': 1.2e-05, 'tensorboard': False, 'flush_freq': True, 'pred_metrics': ['mi_acc'], 'profile_metrics': ['doa'], 'num_responses': 12, 'low_mem': False, 'n_query': 6, 'CDM': 'impact', 'd_in': 4} ####


In [15]:
i_fold = 0
## Dataframe columns : (user_id, question_id, response, category_id)
train_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_train_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
valid_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_valid_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})
test_df = pd.read_csv(
    f'../datasets/2-preprocessed_data/{config["dataset_name"]}_test_{i_fold}.csv',
    encoding='utf-8', dtype={'student_id': int, 'item_id': int, "correct": float,
                                                             "dimension_id": int})

In [16]:
reload(dataset)
train_data = dataset.CATDataset(train_df, concept_map, metadata, config, config['batch_size'])
valid_data = dataset.evalDataset(valid_df, concept_map, metadata, config, batch_size=10000, pin_memory=True)
test_data = dataset.evalDataset(test_df, concept_map, metadata, config, batch_size=10000, pin_memory=True)

/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:150: UserWarning: The torch_array contains both query and the meta set !
  warnings.warn("The torch_array contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:155: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:160: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")


In [17]:
reload(utils)
reload(selectionStrategy)
reload(dataset)
reload(CDM)

S = selectionStrategy.Random(**config)
S.train(train_data, valid_data)

RandomModel()
[INFO 19:46] train on cuda
[INFO 19:46] -- START Training --


/home/arthurb/Programmation/liriscat/liriscat/selectionStrategy/random.py:37: UserWarning: get_params() Notimplemented
  warnings.warn('get_params() Notimplemented')
0it [00:06, ?it/s]


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: CUDA out of memory. Tried to allocate 852.00 MiB. GPU 0 has a total capacity of 3.71 GiB of which 700.69 MiB is free. Including non-PyTorch memory, this process has 1.53 GiB memory in use. Of the allocated memory 1.35 GiB is allocated by PyTorch, and 87.89 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)



In [12]:
test_data.split_query_meta(2)
S.evaluate_test(test_data)

/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:155: UserWarning: The log_tensor contains both query and the meta set !
  warnings.warn("The log_tensor contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/liriscat/dataset/dataset.py:160: UserWarning: The user_dict contains both query and the meta set !
  warnings.warn("The user_dict contains both query and the meta set !")
/home/arthurb/Programmation/liriscat/y/envs/liriscat-env/lib/python3.12/site-packages/IMPACT/model/abstract_model.py:760: UserWarning: The model must be trained before getting user embeddings
  warnings.warn("The model must be trained before getting user embeddings")


({0: {'mi_acc': 0.5768808722496033},
  1: {'mi_acc': 0.5768808722496033},
  2: {'mi_acc': 0.5768808722496033},
  3: {'mi_acc': 0.5768808722496033}},
 {0: {'doa': np.float64(0.5215578899664385)},
  1: {'doa': np.float64(0.5181574837633228)},
  2: {'doa': np.float64(0.5260895961897792)},
  3: {'doa': np.float64(0.5245281427563094)}})

In [ ]:
print('test')